In [22]:
# Copyright 2016, Yarin Gal, All rights reserved.
# This code is based on the code by Jose Miguel Hernandez-Lobato used for his 
# paper "Probabilistic Backpropagation for Scalable Learning of Bayesian Neural Networks".

import warnings
warnings.filterwarnings("ignore")

import math
from scipy.special import logsumexp
import numpy as np

from keras.regularizers import l2
# from keras import Input
from keras.layers import Input
from keras.layers import Dropout
from keras.layers import Dense
# from keras import Model

import time

import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn as nn
import torch.utils.data as Data
from scipy.stats import norm

import time


class netWBB:
    n_bootstrap_samples = 100     #number of bootstrap sampling
    NNs = []

    def __init__(self, X_train, y_train, n_hidden, n_epochs = 40,
        normalize = False):

        """
            Constructor for the class implementing a Bayesian neural network
            trained with the probabilistic back propagation method.

            @param X_train      Matrix with the features for the training data.
            @param y_train      Vector with the target variables for the
                                training data.
            @param n_hidden     Vector with the number of neurons for each
                                hidden layer.
            @param n_epochs     Numer of epochs for which to train the
                                network. The recommended value 40 should be
                                enough.
            @param normalize    Whether to normalize the input features. This
                                is recommended unles the input vector is for
                                example formed by binary features (a
                                fingerprint). In that case we do not recommend
                                to normalize the features.
        """


        # We normalize the training data to have zero mean and unit standard
        # deviation in the training set if necessary
        print("running")
        if normalize:
            self.std_X_train = np.std(X_train, 0)
            self.std_X_train[ self.std_X_train == 0 ] = 1
            self.mean_X_train = np.mean(X_train, 0)
        else:
            self.std_X_train = np.ones(X_train.shape[ 1 ])
            self.mean_X_train = np.zeros(X_train.shape[ 1 ])

        X_train = (X_train - np.full(X_train.shape, self.mean_X_train)) / \
            np.full(X_train.shape, self.std_X_train)

        self.mean_y_train = np.mean(y_train)
        self.std_y_train = np.std(y_train)

        y_train_normalized = (y_train - self.mean_y_train) / self.std_y_train
        y_train_normalized = np.array(y_train_normalized, ndmin = 2).T

        
        # TODO: implement WBB network
        input_shape = X_train.shape[1]
        output_shape = y_train_normalized.shape[1]
        max_index = X_train.shape[0]
        print("shape of input, output and max_index", input_shape, output_shape, max_index)

        lamda = 0.001
        Exp = torch.distributions.exponential.Exponential(torch.tensor([1.0]))
        for m in range (self.n_bootstrap_samples):
            print("Trainning model: ", m)
            weights = [Exp.sample() for _ in range(X_train.shape[0] + 1)]
            print(len(weights))

            # Network with a hidden layer and ReLU activation
            net = torch.nn.Sequential(
            torch.nn.Linear(input_shape, n_hidden[0]),
            torch.nn.ReLU(),
            torch.nn.Linear(n_hidden[0], output_shape),
            )
            
            
            BATCH_SIZE = 64
            optimizer = torch.optim.Adam(net.parameters(), lr=0.01)

            # Minimizing the negative log-likelihood of our data with respect to θ is equivalent to 
            # minimizing the mean squared error between the observed y and our prediction thereof
            loss_func = torch.nn.MSELoss(reduction = 'none')
            torch_dataset = Data.TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train_normalized))

            loader = Data.DataLoader(
            dataset=torch_dataset, 
            batch_size=BATCH_SIZE, 
            shuffle=False, num_workers=2,)

            start_time = time.time()
            for epoch in range(n_epochs):
                optimizer.zero_grad()
                loss = 0

                index1 = 0;
                for step, (batch_x, batch_y) in enumerate(loader):  
                    
                    index2 = (step+1)*BATCH_SIZE   # set index of weight array to multiply with loss: weight*loss_func
                    index2 = min(index2,max_index) 
                    b_x = Variable(batch_x)
                    b_y = Variable(batch_y)               
                    prediction = net(b_x.float())    
                    
                    # print("weight ",(torch.FloatTensor(weights[index1:index2]).shape))
                    # print("loss ", loss_func(prediction, b_y.float()).shape)
                    loss += (torch.FloatTensor(weights[index1:index2]).unsqueeze(1) * loss_func(prediction, b_y.float())).sum()
                    # print((torch.FloatTensor(weights[index1:index2]).unsqueeze(1) * loss_func(prediction, b_y.float())).shape)
                    index1 = index2   

                # #add L1 regularization
                # l1 = 0
                # for p in net.parameters():
                #     l1 = l1 + p.abs().sum()       
                # loss += weights[-1] * lamda * l1

                #add L2 regularization
                l2 = 0
                for p in net.parameters():
                    l2 = l2 + 0.5 * (p ** 2).sum()      
                loss += (weights[-1] * lamda * l2).sum()
                
                loss.backward()         
                optimizer.step()        
                print("EPOCH: ", epoch, " LOSS: ", loss)
            print("BATCH_SIZE: ", BATCH_SIZE, "time execution for a neural net: ", time.time() - start_time )
            self.NNs.append(net)


    def predict(self, X_test, y_test):

        """
            Function for making predictions with the Bayesian neural network.

            @param X_test   The matrix of features for the test data
            
    
            @return m       The predictive mean for the test target variables.
            @return v       The predictive variance for the test target
                            variables.
            @return v_noise The estimated variance for the additive noise.

        """

        X_test = np.array(X_test, ndmin = 2)
        y_test = np.array(y_test, ndmin = 2).T

        # We normalize the test set

        X_test = (X_test - np.full(X_test.shape, self.mean_X_train)) / \
            np.full(X_test.shape, self.std_X_train)

        # We compute the predictive mean and variance for the target variables
        # of the test data

        NN_pred = np.array([self.NNs[m](torch.from_numpy(X_test).float()).detach().numpy() for m in range(self.n_bootstrap_samples)])
        NN_pred = NN_pred * self.std_y_train + self.mean_y_train

        standard_pred = NN_pred[0]
        rmse_standard_pred = np.mean((y_test.squeeze() - standard_pred.squeeze())**2.)**0.5

        # MC_error
        MC_pred = np.mean(NN_pred, axis = 0)
        rmse = np.mean((y_test.squeeze() - MC_pred.squeeze())**2.)**0.5

        variance_ll = 0.1
        tau = 1. / variance_ll
        # T = y_test.shape[0]
        T = self.n_bootstrap_samples
        # We compute the test log-likelihood
        ll = (logsumexp(-0.5 * tau * (y_test[None] - NN_pred)**2., 0) - np.log(T) 
            - 0.5*np.log(2*np.pi) + 0.5*np.log(tau))
        test_ll = np.mean(ll)

        # We are done!
        return rmse_standard_pred, rmse, test_ll


In [23]:
# Copyright 2016, Yarin Gal, All rights reserved.
# This code is based on the code by Jose Miguel Hernandez-Lobato used for his 
# paper "Probabilistic Backpropagation for Scalable Learning of Bayesian Neural Networks".

# This file contains code to train dropout networks on the UCI datasets using the following algorithm:
# 1. Create 20 random splits of the training-test dataset.
# 2. For each split:
# 3.   Create a validation (val) set taking 20% of the training set.
# 4.   Get best hyperparameters: dropout_rate and tau by training on (train-val) set and testing on val set.
# 5.   Train a network on the entire training set with the best pair of hyperparameters.
# 6.   Get the performance (MC RMSE and log-likelihood) on the test set.
# 7. Report the averaged performance (Monte Carlo RMSE and log-likelihood) on all 20 splits.

import math
import numpy as np
import argparse
import sys

# parser=argparse.ArgumentParser()

# parser.add_argument('--dir', '-d', required=True, help='Name of the UCI Dataset directory. Eg: bostonHousing')
# parser.add_argument('--epochx','-e', default=500, type=int, help='Multiplier for the number of epochs for training.')
# parser.add_argument('--hidden', '-nh', default=2, type=int, help='Number of hidden layers for the neural net')

# args=parser.parse_args()

# data_directory = args.dir
# epochs_multiplier = args.epochx
# num_hidden_layers = args.hidden

data_directory = "yacht"
epochs_multiplier = 30
num_hidden_layers = 1

# sys.path.append('net/')

# import netWBB
# from net import netWBB

# We delete previous results

from subprocess import call


_RESULTS_VALIDATION_LL = "./UCI_Datasets/" + data_directory + "/results/validation_ll_" + str(epochs_multiplier) + "_xepochs_" + str(num_hidden_layers) + "_hidden_layers.txt"
_RESULTS_VALIDATION_RMSE = "./UCI_Datasets/" + data_directory + "/results/validation_rmse_" + str(epochs_multiplier) + "_xepochs_" + str(num_hidden_layers) + "_hidden_layers.txt"
_RESULTS_VALIDATION_MC_RMSE = "./UCI_Datasets/" + data_directory + "/results/validation_MC_rmse_" + str(epochs_multiplier) + "_xepochs_" + str(num_hidden_layers) + "_hidden_layers.txt"

_RESULTS_TEST_LL = "./UCI_Datasets/" + data_directory + "/results/test_ll_" + str(epochs_multiplier) + "_xepochs_" + str(num_hidden_layers) + "_hidden_layers.txt"
_RESULTS_TEST_TAU = "./UCI_Datasets/" + data_directory + "/results/test_tau_" + str(epochs_multiplier) + "_xepochs_" + str(num_hidden_layers) + "_hidden_layers.txt"
_RESULTS_TEST_RMSE = "./UCI_Datasets/" + data_directory + "/results/test_rmse_" + str(epochs_multiplier) + "_xepochs_" + str(num_hidden_layers) + "_hidden_layers.txt"
_RESULTS_TEST_MC_RMSE = "./UCI_Datasets/" + data_directory + "/results/test_MC_rmse_" + str(epochs_multiplier) + "_xepochs_" + str(num_hidden_layers) + "_hidden_layers.txt"
_RESULTS_TEST_LOG = "./UCI_Datasets/" + data_directory + "/results/log_" + str(epochs_multiplier) + "_xepochs_" + str(num_hidden_layers) + "_hidden_layers.txt"

_DATA_DIRECTORY_PATH = "./UCI_Datasets/" + data_directory + "/data/"
_DROPOUT_RATES_FILE = _DATA_DIRECTORY_PATH + "dropout_rates.txt"
_TAU_VALUES_FILE = _DATA_DIRECTORY_PATH + "tau_values.txt"
_DATA_FILE = _DATA_DIRECTORY_PATH + "data.txt"
_HIDDEN_UNITS_FILE = _DATA_DIRECTORY_PATH + "n_hidden.txt"
_EPOCHS_FILE = _DATA_DIRECTORY_PATH + "n_epochs.txt"
_INDEX_FEATURES_FILE = _DATA_DIRECTORY_PATH + "index_features.txt"
_INDEX_TARGET_FILE = _DATA_DIRECTORY_PATH + "index_target.txt"
_N_SPLITS_FILE = _DATA_DIRECTORY_PATH + "n_splits.txt"

def _get_index_train_test_path(split_num, train = True):
    """
       Method to generate the path containing the training/test split for the given
       split number (generally from 1 to 20).
       @param split_num      Split number for which the data has to be generated
       @param train          Is true if the data is training data. Else false.
       @return path          Path of the file containing the requried data
    """
    if train:
        return _DATA_DIRECTORY_PATH + "index_train_" + str(split_num) + ".txt"
    else:
        return _DATA_DIRECTORY_PATH + "index_test_" + str(split_num) + ".txt" 


print ("Removing existing result files...")
call(["rm", _RESULTS_VALIDATION_LL])
call(["rm", _RESULTS_VALIDATION_RMSE])
call(["rm", _RESULTS_VALIDATION_MC_RMSE])
call(["rm", _RESULTS_TEST_LL])
call(["rm", _RESULTS_TEST_TAU])
call(["rm", _RESULTS_TEST_RMSE])
call(["rm", _RESULTS_TEST_MC_RMSE])
call(["rm", _RESULTS_TEST_LOG])
print ("Result files removed.")

# We fix the random seed

np.random.seed(1)

print ("Loading data and other hyperparameters...")
# We load the data

data = np.loadtxt(_DATA_FILE)

# We load the number of hidden units

n_hidden = np.loadtxt(_HIDDEN_UNITS_FILE).tolist()

# We load the number of training epocs

n_epochs = np.loadtxt(_EPOCHS_FILE).tolist()

# We load the indexes for the features and for the target

index_features = np.loadtxt(_INDEX_FEATURES_FILE)
index_target = np.loadtxt(_INDEX_TARGET_FILE)

X = data[ : , [int(i) for i in index_features.tolist()] ]
y = data[ : , int(index_target.tolist()) ]

# We iterate over the training test splits

n_splits = np.loadtxt(_N_SPLITS_FILE)
print ("Done.")

errors, MC_errors, lls = [], [], []
# for split in range(int(n_splits)):
for split in range(5):  # first, test in 5 splits dataset
    print ("Trainning on split: ", split)
    # We load the indexes of the training and test sets
    print ('Loading file: ' + _get_index_train_test_path(split, train=True))
    print ('Loading file: ' + _get_index_train_test_path(split, train=False))
    index_train = np.loadtxt(_get_index_train_test_path(split, train=True))
    index_test = np.loadtxt(_get_index_train_test_path(split, train=False))

    X_train = X[ [int(i) for i in index_train.tolist()] ]
    y_train = y[ [int(i) for i in index_train.tolist()] ]
    
    X_test = X[ [int(i) for i in index_test.tolist()] ]
    y_test = y[ [int(i) for i in index_test.tolist()] ]

    X_train_original = X_train
    y_train_original = y_train
    num_training_examples = int(0.8 * X_train.shape[0])
    X_validation = X_train[num_training_examples:, :]
    y_validation = y_train[num_training_examples:]
    X_train = X_train[0:num_training_examples, :]
    y_train = y_train[0:num_training_examples]
    
    # Printing the size of the training, validation and test sets
    print ('Number of training examples: ' + str(X_train.shape[0]))
    print ('Number of validation examples: ' + str(X_validation.shape[0]))
    print ('Number of test examples: ' + str(X_test.shape[0]))
    print ('Number of train_original examples: ' + str(X_train_original.shape[0]))

    # List of hyperparameters which we will try out using grid-search
    # dropout_rates = np.loadtxt(_DROPOUT_RATES_FILE).tolist()
    # tau_values = np.loadtxt(_TAU_VALUES_FILE).tolist()


    network = netWBB(X_train_original, y_train_original, ([ int(n_hidden) ] * num_hidden_layers),
            normalize = True, n_epochs = int(n_epochs * epochs_multiplier))


            
#             # Storing validation results
#             with open(_RESULTS_VALIDATION_RMSE, "a") as myfile:
#                 myfile.write('Dropout_Rate: ' + repr(dropout_rate) + ' Tau: ' + repr(tau) + ' :: ')
#                 myfile.write(repr(error) + '\n')

#             with open(_RESULTS_VALIDATION_MC_RMSE, "a") as myfile:
#                 myfile.write('Dropout_Rate: ' + repr(dropout_rate) + ' Tau: ' + repr(tau) + ' :: ')
#                 myfile.write(repr(MC_error) + '\n')

#             with open(_RESULTS_VALIDATION_LL, "a") as myfile:
#                 myfile.write('Dropout_Rate: ' + repr(dropout_rate) + ' Tau: ' + repr(tau) + ' :: ')
#                 myfile.write(repr(ll) + '\n')

#     # Storing test results
#     best_network = net.net(X_train_original, y_train_original, ([ int(n_hidden) ] * num_hidden_layers),
#                     normalize = True, n_epochs = int(n_epochs * epochs_multiplier), tau = best_tau,
#                     dropout = best_dropout)
    error, MC_error, ll = network.predict(X_test, y_test)
    
    with open(_RESULTS_TEST_RMSE, "a") as myfile:
        myfile.write(repr(error) + '\n')

    with open(_RESULTS_TEST_MC_RMSE, "a") as myfile:
        myfile.write(repr(MC_error) + '\n')

    with open(_RESULTS_TEST_LL, "a") as myfile:
        myfile.write(repr(ll) + '\n')

    # with open(_RESULTS_TEST_TAU, "a") as myfile:
    #     myfile.write(repr(best_network.tau) + '\n')

    print ("Tests on split " + str(split) + " complete.")
    errors += [error]
    MC_errors += [MC_error]
    lls += [ll]

with open(_RESULTS_TEST_LOG, "a") as myfile:
    myfile.write('errors %f +- %f (stddev) +- %f (std error), median %f 25p %f 75p %f \n' % (
        np.mean(errors), np.std(errors), np.std(errors)/math.sqrt(n_splits),
        np.percentile(errors, 50), np.percentile(errors, 25), np.percentile(errors, 75)))
    myfile.write('MC errors %f +- %f (stddev) +- %f (std error), median %f 25p %f 75p %f \n' % (
        np.mean(MC_errors), np.std(MC_errors), np.std(MC_errors)/math.sqrt(n_splits),
        np.percentile(MC_errors, 50), np.percentile(MC_errors, 25), np.percentile(MC_errors, 75)))
    myfile.write('lls %f +- %f (stddev) +- %f (std error), median %f 25p %f 75p %f \n' % (
        np.mean(lls), np.std(lls), np.std(lls)/math.sqrt(n_splits), 
        np.percentile(lls, 50), np.percentile(lls, 25), np.percentile(lls, 75)))


Removing existing result files...
Result files removed.
Loading data and other hyperparameters...
Done.
Trainning on split:  0
Loading file: ./UCI_Datasets/yacht/data/index_train_0.txt
Loading file: ./UCI_Datasets/yacht/data/index_test_0.txt
Number of training examples: 221
Number of validation examples: 56
Number of test examples: 31
Number of train_original examples: 277
running
shape 6 1 277
Trainning model:  0
278
EPOCH:  0  LOSS:  tensor(356.8891, grad_fn=<AddBackward0>)
EPOCH:  1  LOSS:  tensor(307.4855, grad_fn=<AddBackward0>)
EPOCH:  2  LOSS:  tensor(270.0096, grad_fn=<AddBackward0>)
EPOCH:  3  LOSS:  tensor(239.9880, grad_fn=<AddBackward0>)
EPOCH:  4  LOSS:  tensor(214.6577, grad_fn=<AddBackward0>)
EPOCH:  5  LOSS:  tensor(192.3997, grad_fn=<AddBackward0>)
EPOCH:  6  LOSS:  tensor(172.4898, grad_fn=<AddBackward0>)
EPOCH:  7  LOSS:  tensor(154.5892, grad_fn=<AddBackward0>)
EPOCH:  8  LOSS:  tensor(138.7633, grad_fn=<AddBackward0>)
EPOCH:  9  LOSS:  tensor(125.2663, grad_fn=<Add

EPOCH:  133  LOSS:  tensor(1.2032, grad_fn=<AddBackward0>)
EPOCH:  134  LOSS:  tensor(1.1892, grad_fn=<AddBackward0>)
EPOCH:  135  LOSS:  tensor(1.1752, grad_fn=<AddBackward0>)
EPOCH:  136  LOSS:  tensor(1.1610, grad_fn=<AddBackward0>)
EPOCH:  137  LOSS:  tensor(1.1481, grad_fn=<AddBackward0>)
EPOCH:  138  LOSS:  tensor(1.1356, grad_fn=<AddBackward0>)
EPOCH:  139  LOSS:  tensor(1.1234, grad_fn=<AddBackward0>)
EPOCH:  140  LOSS:  tensor(1.1120, grad_fn=<AddBackward0>)
EPOCH:  141  LOSS:  tensor(1.1007, grad_fn=<AddBackward0>)
EPOCH:  142  LOSS:  tensor(1.0896, grad_fn=<AddBackward0>)
EPOCH:  143  LOSS:  tensor(1.0787, grad_fn=<AddBackward0>)
EPOCH:  144  LOSS:  tensor(1.0679, grad_fn=<AddBackward0>)
EPOCH:  145  LOSS:  tensor(1.0577, grad_fn=<AddBackward0>)
EPOCH:  146  LOSS:  tensor(1.0479, grad_fn=<AddBackward0>)
EPOCH:  147  LOSS:  tensor(1.0384, grad_fn=<AddBackward0>)
EPOCH:  148  LOSS:  tensor(1.0291, grad_fn=<AddBackward0>)
EPOCH:  149  LOSS:  tensor(1.0198, grad_fn=<AddBackward0

EPOCH:  272  LOSS:  tensor(0.4277, grad_fn=<AddBackward0>)
EPOCH:  273  LOSS:  tensor(0.4255, grad_fn=<AddBackward0>)
EPOCH:  274  LOSS:  tensor(0.4233, grad_fn=<AddBackward0>)
EPOCH:  275  LOSS:  tensor(0.4212, grad_fn=<AddBackward0>)
EPOCH:  276  LOSS:  tensor(0.4193, grad_fn=<AddBackward0>)
EPOCH:  277  LOSS:  tensor(0.4173, grad_fn=<AddBackward0>)
EPOCH:  278  LOSS:  tensor(0.4154, grad_fn=<AddBackward0>)
EPOCH:  279  LOSS:  tensor(0.4136, grad_fn=<AddBackward0>)
EPOCH:  280  LOSS:  tensor(0.4118, grad_fn=<AddBackward0>)
EPOCH:  281  LOSS:  tensor(0.4101, grad_fn=<AddBackward0>)
EPOCH:  282  LOSS:  tensor(0.4083, grad_fn=<AddBackward0>)
EPOCH:  283  LOSS:  tensor(0.4067, grad_fn=<AddBackward0>)
EPOCH:  284  LOSS:  tensor(0.4050, grad_fn=<AddBackward0>)
EPOCH:  285  LOSS:  tensor(0.4032, grad_fn=<AddBackward0>)
EPOCH:  286  LOSS:  tensor(0.4013, grad_fn=<AddBackward0>)
EPOCH:  287  LOSS:  tensor(0.3996, grad_fn=<AddBackward0>)
EPOCH:  288  LOSS:  tensor(0.3979, grad_fn=<AddBackward0

EPOCH:  411  LOSS:  tensor(0.2394, grad_fn=<AddBackward0>)
EPOCH:  412  LOSS:  tensor(0.2385, grad_fn=<AddBackward0>)
EPOCH:  413  LOSS:  tensor(0.2377, grad_fn=<AddBackward0>)
EPOCH:  414  LOSS:  tensor(0.2368, grad_fn=<AddBackward0>)
EPOCH:  415  LOSS:  tensor(0.2360, grad_fn=<AddBackward0>)
EPOCH:  416  LOSS:  tensor(0.2352, grad_fn=<AddBackward0>)
EPOCH:  417  LOSS:  tensor(0.2344, grad_fn=<AddBackward0>)
EPOCH:  418  LOSS:  tensor(0.2336, grad_fn=<AddBackward0>)
EPOCH:  419  LOSS:  tensor(0.2328, grad_fn=<AddBackward0>)
EPOCH:  420  LOSS:  tensor(0.2320, grad_fn=<AddBackward0>)
EPOCH:  421  LOSS:  tensor(0.2313, grad_fn=<AddBackward0>)
EPOCH:  422  LOSS:  tensor(0.2304, grad_fn=<AddBackward0>)
EPOCH:  423  LOSS:  tensor(0.2297, grad_fn=<AddBackward0>)
EPOCH:  424  LOSS:  tensor(0.2288, grad_fn=<AddBackward0>)
EPOCH:  425  LOSS:  tensor(0.2280, grad_fn=<AddBackward0>)
EPOCH:  426  LOSS:  tensor(0.2273, grad_fn=<AddBackward0>)
EPOCH:  427  LOSS:  tensor(0.2265, grad_fn=<AddBackward0

EPOCH:  550  LOSS:  tensor(0.1619, grad_fn=<AddBackward0>)
EPOCH:  551  LOSS:  tensor(0.1615, grad_fn=<AddBackward0>)
EPOCH:  552  LOSS:  tensor(0.1612, grad_fn=<AddBackward0>)
EPOCH:  553  LOSS:  tensor(0.1607, grad_fn=<AddBackward0>)
EPOCH:  554  LOSS:  tensor(0.1603, grad_fn=<AddBackward0>)
EPOCH:  555  LOSS:  tensor(0.1599, grad_fn=<AddBackward0>)
EPOCH:  556  LOSS:  tensor(0.1595, grad_fn=<AddBackward0>)
EPOCH:  557  LOSS:  tensor(0.1591, grad_fn=<AddBackward0>)
EPOCH:  558  LOSS:  tensor(0.1587, grad_fn=<AddBackward0>)
EPOCH:  559  LOSS:  tensor(0.1583, grad_fn=<AddBackward0>)
EPOCH:  560  LOSS:  tensor(0.1579, grad_fn=<AddBackward0>)
EPOCH:  561  LOSS:  tensor(0.1575, grad_fn=<AddBackward0>)
EPOCH:  562  LOSS:  tensor(0.1570, grad_fn=<AddBackward0>)
EPOCH:  563  LOSS:  tensor(0.1567, grad_fn=<AddBackward0>)
EPOCH:  564  LOSS:  tensor(0.1563, grad_fn=<AddBackward0>)
EPOCH:  565  LOSS:  tensor(0.1559, grad_fn=<AddBackward0>)
EPOCH:  566  LOSS:  tensor(0.1555, grad_fn=<AddBackward0

Exception ignored in: <function WeakValueDictionary.__init__.<locals>.remove at 0x102ba5a60>
Traceback (most recent call last):
  File "/Users/mac/anaconda3/envs/SSL/lib/python3.6/weakref.py", line 109, in remove
    def remove(wr, selfref=ref(self), _atomic_removal=_remove_dead_weakref):
KeyboardInterrupt


EPOCH:  626  LOSS:  tensor(0.1370, grad_fn=<AddBackward0>)
EPOCH:  627  LOSS:  tensor(0.1367, grad_fn=<AddBackward0>)
EPOCH:  628  LOSS:  tensor(0.1365, grad_fn=<AddBackward0>)
EPOCH:  629  LOSS:  tensor(0.1362, grad_fn=<AddBackward0>)
EPOCH:  630  LOSS:  tensor(0.1359, grad_fn=<AddBackward0>)
EPOCH:  631  LOSS:  tensor(0.1357, grad_fn=<AddBackward0>)
EPOCH:  632  LOSS:  tensor(0.1354, grad_fn=<AddBackward0>)
EPOCH:  633  LOSS:  tensor(0.1352, grad_fn=<AddBackward0>)
EPOCH:  634  LOSS:  tensor(0.1349, grad_fn=<AddBackward0>)
EPOCH:  635  LOSS:  tensor(0.1347, grad_fn=<AddBackward0>)
EPOCH:  636  LOSS:  tensor(0.1344, grad_fn=<AddBackward0>)
EPOCH:  637  LOSS:  tensor(0.1342, grad_fn=<AddBackward0>)
EPOCH:  638  LOSS:  tensor(0.1339, grad_fn=<AddBackward0>)
EPOCH:  639  LOSS:  tensor(0.1337, grad_fn=<AddBackward0>)
EPOCH:  640  LOSS:  tensor(0.1334, grad_fn=<AddBackward0>)
EPOCH:  641  LOSS:  tensor(0.1332, grad_fn=<AddBackward0>)
EPOCH:  642  LOSS:  tensor(0.1329, grad_fn=<AddBackward0

KeyboardInterrupt: 

In [13]:
# Copyright 2016, Yarin Gal, All rights reserved.
# This code is based on the code by Jose Miguel Hernandez-Lobato used for his 
# paper "Probabilistic Backpropagation for Scalable Learning of Bayesian Neural Networks".

import warnings
warnings.filterwarnings("ignore")

import math
from scipy.special import logsumexp
import numpy as np

from keras.regularizers import l2
# from keras import Input
from keras.layers import Input
from keras.layers import Dropout
from keras.layers import Dense
# from keras import Model

import time

import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn as nn
import torch.utils.data as Data
from scipy.stats import norm

import time

In [17]:
class netWBB:
    n_bootstrap_samples = 100     #number of bootstrap sampling
    NNs = []

    def __init__(self, X_train, y_train, n_hidden, n_epochs = 40,
        normalize = False):

        """
            Constructor for the class implementing a Bayesian neural network
            trained with the probabilistic back propagation method.

            @param X_train      Matrix with the features for the training data.
            @param y_train      Vector with the target variables for the
                                training data.
            @param n_hidden     Vector with the number of neurons for each
                                hidden layer.
            @param n_epochs     Numer of epochs for which to train the
                                network. The recommended value 40 should be
                                enough.
            @param normalize    Whether to normalize the input features. This
                                is recommended unles the input vector is for
                                example formed by binary features (a
                                fingerprint). In that case we do not recommend
                                to normalize the features.
        """


        # We normalize the training data to have zero mean and unit standard
        # deviation in the training set if necessary
        print("running")
        if normalize:
            self.std_X_train = np.std(X_train, 0)
            self.std_X_train[ self.std_X_train == 0 ] = 1
            self.mean_X_train = np.mean(X_train, 0)
        else:
            self.std_X_train = np.ones(X_train.shape[ 1 ])
            self.mean_X_train = np.zeros(X_train.shape[ 1 ])

        X_train = (X_train - np.full(X_train.shape, self.mean_X_train)) / \
            np.full(X_train.shape, self.std_X_train)

        self.mean_y_train = np.mean(y_train)
        self.std_y_train = np.std(y_train)

        y_train_normalized = (y_train - self.mean_y_train) / self.std_y_train
        y_train_normalized = np.array(y_train_normalized, ndmin = 2).T

        
        # TODO: implement WBB network
        input_shape = X_train.shape[1]
        output_shape = y_train_normalized.shape[1]
        max_index = X_train.shape[0]
        print("shape of input, output and max_index", input_shape, output_shape, max_index)

        lamda = 0.001
        Exp = torch.distributions.exponential.Exponential(torch.tensor([1.0]))
        for m in range (self.n_bootstrap_samples):
            print("Trainning model: ", m)
            weights = [Exp.sample() for _ in range(X_train.shape[0] + 1)]
            print("....", len(weights))

            # Network with a hidden layer and ReLU activation
            net = torch.nn.Sequential(
            torch.nn.Linear(input_shape, n_hidden[0]),
            torch.nn.ReLU(),
            torch.nn.Linear(n_hidden[0], output_shape),
            )
            
            
            BATCH_SIZE = 64
            optimizer = torch.optim.Adam(net.parameters(), lr=0.01)

            # Minimizing the negative log-likelihood of our data with respect to θ is equivalent to 
            # minimizing the mean squared error between the observed y and our prediction thereof
            loss_func = torch.nn.MSELoss(reduction = 'none')
            torch_dataset = Data.TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train_normalized))

            loader=Data.DataLoader(
            dataset=torch_dataset, 
            batch_size=BATCH_SIZE, 
            shuffle=False, num_workers=2,)

            start_time = time.time()
            for epoch in range(n_epochs):
                optimizer.zero_grad()
                loss = 0

                index1 = 0;
                for step, (batch_x, batch_y) in enumerate(loader):  
                    
                    index2 = (step+1)*BATCH_SIZE   # set index of weight array to multiply with loss: weight*loss_func
                    index2 = min(index2,max_index) 
                    b_x = Variable(batch_x)
                    b_y = Variable(batch_y)               
                    prediction = net(b_x.float())    
                    
                    # print("weight ",(torch.FloatTensor(weights[index1:index2]).shape))
                    # print("loss ", loss_func(prediction, b_y.float()).shape)
                    loss += (torch.FloatTensor(weights[index1:index2]).unsqueeze(1) * loss_func(prediction, b_y.float())).sum()
                    # print((torch.FloatTensor(weights[index1:index2]).unsqueeze(1) * loss_func(prediction, b_y.float())).shape)
                    index1 = index2   

                # #add L1 regularization
                # l1 = 0
                # for p in net.parameters():
                #     l1 = l1 + p.abs().sum()       
                # loss += weights[-1] * lamda * l1

                #add L2 regularization
                l2 = 0
                for p in net.parameters():
                    l2 = l2 + 0.5 * (p ** 2).sum()      
                loss += (weights[-1] * lamda * l2).sum()
                
                loss.backward()         
                optimizer.step()        
                print("EPOCH: ", epoch, " LOSS: ", loss)
            print("BATCH_SIZE: ", BATCH_SIZE, "time execution for a neural net: ", time.time() - start_time )
            self.NNs.append(net)


    def predict(self, X_test, y_test):

        """
            Function for making predictions with the Bayesian neural network.

            @param X_test   The matrix of features for the test data
            
    
            @return m       The predictive mean for the test target variables.
            @return v       The predictive variance for the test target
                            variables.
            @return v_noise The estimated variance for the additive noise.

        """

        X_test = np.array(X_test, ndmin = 2)
        y_test = np.array(y_test, ndmin = 2).T

        # We normalize the test set

        X_test = (X_test - np.full(X_test.shape, self.mean_X_train)) / \
            np.full(X_test.shape, self.std_X_train)

        # We compute the predictive mean and variance for the target variables
        # of the test data

        NN_pred = np.array([self.NNs[m](torch.from_numpy(X_test).float()).detach().numpy() for m in range(self.n_bootstrap_samples)])
        NN_pred = NN_pred * self.std_y_train + self.mean_y_train

        standard_pred = NN_pred[0]
        rmse_standard_pred = np.mean((y_test.squeeze() - standard_pred.squeeze())**2.)**0.5

        # MC_error
        MC_pred = np.mean(NN_pred, axis = 0)
        rmse = np.mean((y_test.squeeze() - MC_pred.squeeze())**2.)**0.5

        variance_ll = 0.1
        tau = 1. / variance_ll
        # T = y_test.shape[0]
        T = self.n_bootstrap_samples
        # We compute the test log-likelihood
        ll = (logsumexp(-0.5 * tau * (y_test[None] - NN_pred)**2., 0) - np.log(T) 
            - 0.5*np.log(2*np.pi) + 0.5*np.log(tau))
        test_ll = np.mean(ll)

        # We are done!
        return rmse_standard_pred, rmse, test_ll


In [18]:
# Copyright 2016, Yarin Gal, All rights reserved.
# This code is based on the code by Jose Miguel Hernandez-Lobato used for his 
# paper "Probabilistic Backpropagation for Scalable Learning of Bayesian Neural Networks".

# This file contains code to train dropout networks on the UCI datasets using the following algorithm:
# 1. Create 20 random splits of the training-test dataset.
# 2. For each split:
# 3.   Create a validation (val) set taking 20% of the training set.
# 4.   Get best hyperparameters: dropout_rate and tau by training on (train-val) set and testing on val set.
# 5.   Train a network on the entire training set with the best pair of hyperparameters.
# 6.   Get the performance (MC RMSE and log-likelihood) on the test set.
# 7. Report the averaged performance (Monte Carlo RMSE and log-likelihood) on all 20 splits.

import math
import numpy as np
import argparse
import sys

# parser=argparse.ArgumentParser()

# parser.add_argument('--dir', '-d', required=True, help='Name of the UCI Dataset directory. Eg: bostonHousing')
# parser.add_argument('--epochx','-e', default=500, type=int, help='Multiplier for the number of epochs for training.')
# parser.add_argument('--hidden', '-nh', default=2, type=int, help='Number of hidden layers for the neural net')

# args=parser.parse_args()

# data_directory = args.dir
# epochs_multiplier = args.epochx
# num_hidden_layers = args.hidden

data_directory = "yacht"
epochs_multiplier = 30
num_hidden_layers = 1

# sys.path.append('net/')

# import netWBB
# from net import netWBB

# We delete previous results

from subprocess import call


_RESULTS_VALIDATION_LL = "./UCI_Datasets/" + data_directory + "/results/validation_ll_" + str(epochs_multiplier) + "_xepochs_" + str(num_hidden_layers) + "_hidden_layers.txt"
_RESULTS_VALIDATION_RMSE = "./UCI_Datasets/" + data_directory + "/results/validation_rmse_" + str(epochs_multiplier) + "_xepochs_" + str(num_hidden_layers) + "_hidden_layers.txt"
_RESULTS_VALIDATION_MC_RMSE = "./UCI_Datasets/" + data_directory + "/results/validation_MC_rmse_" + str(epochs_multiplier) + "_xepochs_" + str(num_hidden_layers) + "_hidden_layers.txt"

_RESULTS_TEST_LL = "./UCI_Datasets/" + data_directory + "/results/test_ll_" + str(epochs_multiplier) + "_xepochs_" + str(num_hidden_layers) + "_hidden_layers.txt"
_RESULTS_TEST_TAU = "./UCI_Datasets/" + data_directory + "/results/test_tau_" + str(epochs_multiplier) + "_xepochs_" + str(num_hidden_layers) + "_hidden_layers.txt"
_RESULTS_TEST_RMSE = "./UCI_Datasets/" + data_directory + "/results/test_rmse_" + str(epochs_multiplier) + "_xepochs_" + str(num_hidden_layers) + "_hidden_layers.txt"
_RESULTS_TEST_MC_RMSE = "./UCI_Datasets/" + data_directory + "/results/test_MC_rmse_" + str(epochs_multiplier) + "_xepochs_" + str(num_hidden_layers) + "_hidden_layers.txt"
_RESULTS_TEST_LOG = "./UCI_Datasets/" + data_directory + "/results/log_" + str(epochs_multiplier) + "_xepochs_" + str(num_hidden_layers) + "_hidden_layers.txt"

_DATA_DIRECTORY_PATH = "./UCI_Datasets/" + data_directory + "/data/"
_DROPOUT_RATES_FILE = _DATA_DIRECTORY_PATH + "dropout_rates.txt"
_TAU_VALUES_FILE = _DATA_DIRECTORY_PATH + "tau_values.txt"
_DATA_FILE = _DATA_DIRECTORY_PATH + "data.txt"
_HIDDEN_UNITS_FILE = _DATA_DIRECTORY_PATH + "n_hidden.txt"
_EPOCHS_FILE = _DATA_DIRECTORY_PATH + "n_epochs.txt"
_INDEX_FEATURES_FILE = _DATA_DIRECTORY_PATH + "index_features.txt"
_INDEX_TARGET_FILE = _DATA_DIRECTORY_PATH + "index_target.txt"
_N_SPLITS_FILE = _DATA_DIRECTORY_PATH + "n_splits.txt"


In [19]:
def _get_index_train_test_path(split_num, train = True):
    """
       Method to generate the path containing the training/test split for the given
       split number (generally from 1 to 20).
       @param split_num      Split number for which the data has to be generated
       @param train          Is true if the data is training data. Else false.
       @return path          Path of the file containing the requried data
    """
    if train:
        return _DATA_DIRECTORY_PATH + "index_train_" + str(split_num) + ".txt"
    else:
        return _DATA_DIRECTORY_PATH + "index_test_" + str(split_num) + ".txt" 


print ("Removing existing result files...")
call(["rm", _RESULTS_VALIDATION_LL])
call(["rm", _RESULTS_VALIDATION_RMSE])
call(["rm", _RESULTS_VALIDATION_MC_RMSE])
call(["rm", _RESULTS_TEST_LL])
call(["rm", _RESULTS_TEST_TAU])
call(["rm", _RESULTS_TEST_RMSE])
call(["rm", _RESULTS_TEST_MC_RMSE])
call(["rm", _RESULTS_TEST_LOG])
print ("Result files removed.")

# We fix the random seed

np.random.seed(1)

print ("Loading data and other hyperparameters...")
# We load the data

data = np.loadtxt(_DATA_FILE)

# We load the number of hidden units

n_hidden = np.loadtxt(_HIDDEN_UNITS_FILE).tolist()

# We load the number of training epocs

n_epochs = np.loadtxt(_EPOCHS_FILE).tolist()

# We load the indexes for the features and for the target

index_features = np.loadtxt(_INDEX_FEATURES_FILE)
index_target = np.loadtxt(_INDEX_TARGET_FILE)

X = data[ : , [int(i) for i in index_features.tolist()] ]
y = data[ : , int(index_target.tolist()) ]

# We iterate over the training test splits

n_splits = np.loadtxt(_N_SPLITS_FILE)
print ("Done.")

errors, MC_errors, lls = [], [], []
# for split in range(int(n_splits)):
for split in range(5):  # first, test in 5 splits dataset
    print ("Trainning on split: ", split)
    # We load the indexes of the training and test sets
    print ('Loading file: ' + _get_index_train_test_path(split, train=True))
    print ('Loading file: ' + _get_index_train_test_path(split, train=False))
    index_train = np.loadtxt(_get_index_train_test_path(split, train=True))
    index_test = np.loadtxt(_get_index_train_test_path(split, train=False))

    X_train = X[ [int(i) for i in index_train.tolist()] ]
    y_train = y[ [int(i) for i in index_train.tolist()] ]
    
    X_test = X[ [int(i) for i in index_test.tolist()] ]
    y_test = y[ [int(i) for i in index_test.tolist()] ]

    X_train_original = X_train
    y_train_original = y_train
    num_training_examples = int(0.8 * X_train.shape[0])
    X_validation = X_train[num_training_examples:, :]
    y_validation = y_train[num_training_examples:]
    X_train = X_train[0:num_training_examples, :]
    y_train = y_train[0:num_training_examples]
    
    # Printing the size of the training, validation and test sets
    print ('Number of training examples: ' + str(X_train.shape[0]))
    print ('Number of validation examples: ' + str(X_validation.shape[0]))
    print ('Number of test examples: ' + str(X_test.shape[0]))
    print ('Number of train_original examples: ' + str(X_train_original.shape[0]))

    # List of hyperparameters which we will try out using grid-search
    # dropout_rates = np.loadtxt(_DROPOUT_RATES_FILE).tolist()
    # tau_values = np.loadtxt(_TAU_VALUES_FILE).tolist()


    network = netWBB(X_train_original, y_train_original, ([ int(n_hidden) ] * num_hidden_layers),
            normalize = True, n_epochs = int(n_epochs * epochs_multiplier))


            
#             # Storing validation results
#             with open(_RESULTS_VALIDATION_RMSE, "a") as myfile:
#                 myfile.write('Dropout_Rate: ' + repr(dropout_rate) + ' Tau: ' + repr(tau) + ' :: ')
#                 myfile.write(repr(error) + '\n')

#             with open(_RESULTS_VALIDATION_MC_RMSE, "a") as myfile:
#                 myfile.write('Dropout_Rate: ' + repr(dropout_rate) + ' Tau: ' + repr(tau) + ' :: ')
#                 myfile.write(repr(MC_error) + '\n')

#             with open(_RESULTS_VALIDATION_LL, "a") as myfile:
#                 myfile.write('Dropout_Rate: ' + repr(dropout_rate) + ' Tau: ' + repr(tau) + ' :: ')
#                 myfile.write(repr(ll) + '\n')

#     # Storing test results
#     best_network = net.net(X_train_original, y_train_original, ([ int(n_hidden) ] * num_hidden_layers),
#                     normalize = True, n_epochs = int(n_epochs * epochs_multiplier), tau = best_tau,
#                     dropout = best_dropout)
    error, MC_error, ll = network.predict(X_test, y_test)
    
    with open(_RESULTS_TEST_RMSE, "a") as myfile:
        myfile.write(repr(error) + '\n')

    with open(_RESULTS_TEST_MC_RMSE, "a") as myfile:
        myfile.write(repr(MC_error) + '\n')

    with open(_RESULTS_TEST_LL, "a") as myfile:
        myfile.write(repr(ll) + '\n')

    # with open(_RESULTS_TEST_TAU, "a") as myfile:
    #     myfile.write(repr(best_network.tau) + '\n')

    print ("Tests on split " + str(split) + " complete.")
    errors += [error]
    MC_errors += [MC_error]
    lls += [ll]

with open(_RESULTS_TEST_LOG, "a") as myfile:
    myfile.write('errors %f +- %f (stddev) +- %f (std error), median %f 25p %f 75p %f \n' % (
        np.mean(errors), np.std(errors), np.std(errors)/math.sqrt(n_splits),
        np.percentile(errors, 50), np.percentile(errors, 25), np.percentile(errors, 75)))
    myfile.write('MC errors %f +- %f (stddev) +- %f (std error), median %f 25p %f 75p %f \n' % (
        np.mean(MC_errors), np.std(MC_errors), np.std(MC_errors)/math.sqrt(n_splits),
        np.percentile(MC_errors, 50), np.percentile(MC_errors, 25), np.percentile(MC_errors, 75)))
    myfile.write('lls %f +- %f (stddev) +- %f (std error), median %f 25p %f 75p %f \n' % (
        np.mean(lls), np.std(lls), np.std(lls)/math.sqrt(n_splits), 
        np.percentile(lls, 50), np.percentile(lls, 25), np.percentile(lls, 75)))


Removing existing result files...
Result files removed.
Loading data and other hyperparameters...
Done.
Trainning on split:  0
Loading file: ./UCI_Datasets/yacht/data/index_train_0.txt
Loading file: ./UCI_Datasets/yacht/data/index_test_0.txt
Number of training examples: 221
Number of validation examples: 56
Number of test examples: 31
Number of train_original examples: 277
running
shape of input, output and max_index 6 1 277
Trainning model:  0
.... 278
EPOCH:  0  LOSS:  tensor(373.3935, grad_fn=<AddBackward0>)
EPOCH:  1  LOSS:  tensor(332.3792, grad_fn=<AddBackward0>)
EPOCH:  2  LOSS:  tensor(297.7012, grad_fn=<AddBackward0>)
EPOCH:  3  LOSS:  tensor(267.3447, grad_fn=<AddBackward0>)
EPOCH:  4  LOSS:  tensor(240.2691, grad_fn=<AddBackward0>)
EPOCH:  5  LOSS:  tensor(216.2865, grad_fn=<AddBackward0>)
EPOCH:  6  LOSS:  tensor(195.3037, grad_fn=<AddBackward0>)
EPOCH:  7  LOSS:  tensor(177.3007, grad_fn=<AddBackward0>)
EPOCH:  8  LOSS:  tensor(162.1440, grad_fn=<AddBackward0>)
EPOCH:  9  

EPOCH:  132  LOSS:  tensor(1.5608, grad_fn=<AddBackward0>)
EPOCH:  133  LOSS:  tensor(1.5247, grad_fn=<AddBackward0>)
EPOCH:  134  LOSS:  tensor(1.4897, grad_fn=<AddBackward0>)
EPOCH:  135  LOSS:  tensor(1.4564, grad_fn=<AddBackward0>)
EPOCH:  136  LOSS:  tensor(1.4240, grad_fn=<AddBackward0>)
EPOCH:  137  LOSS:  tensor(1.3924, grad_fn=<AddBackward0>)
EPOCH:  138  LOSS:  tensor(1.3620, grad_fn=<AddBackward0>)
EPOCH:  139  LOSS:  tensor(1.3327, grad_fn=<AddBackward0>)
EPOCH:  140  LOSS:  tensor(1.3049, grad_fn=<AddBackward0>)
EPOCH:  141  LOSS:  tensor(1.2781, grad_fn=<AddBackward0>)
EPOCH:  142  LOSS:  tensor(1.2521, grad_fn=<AddBackward0>)
EPOCH:  143  LOSS:  tensor(1.2262, grad_fn=<AddBackward0>)
EPOCH:  144  LOSS:  tensor(1.2014, grad_fn=<AddBackward0>)
EPOCH:  145  LOSS:  tensor(1.1781, grad_fn=<AddBackward0>)
EPOCH:  146  LOSS:  tensor(1.1553, grad_fn=<AddBackward0>)
EPOCH:  147  LOSS:  tensor(1.1329, grad_fn=<AddBackward0>)
EPOCH:  148  LOSS:  tensor(1.1109, grad_fn=<AddBackward0

EPOCH:  271  LOSS:  tensor(0.2967, grad_fn=<AddBackward0>)
EPOCH:  272  LOSS:  tensor(0.2945, grad_fn=<AddBackward0>)
EPOCH:  273  LOSS:  tensor(0.2926, grad_fn=<AddBackward0>)
EPOCH:  274  LOSS:  tensor(0.2906, grad_fn=<AddBackward0>)
EPOCH:  275  LOSS:  tensor(0.2886, grad_fn=<AddBackward0>)
EPOCH:  276  LOSS:  tensor(0.2866, grad_fn=<AddBackward0>)
EPOCH:  277  LOSS:  tensor(0.2847, grad_fn=<AddBackward0>)
EPOCH:  278  LOSS:  tensor(0.2826, grad_fn=<AddBackward0>)
EPOCH:  279  LOSS:  tensor(0.2808, grad_fn=<AddBackward0>)
EPOCH:  280  LOSS:  tensor(0.2789, grad_fn=<AddBackward0>)
EPOCH:  281  LOSS:  tensor(0.2771, grad_fn=<AddBackward0>)
EPOCH:  282  LOSS:  tensor(0.2753, grad_fn=<AddBackward0>)
EPOCH:  283  LOSS:  tensor(0.2736, grad_fn=<AddBackward0>)
EPOCH:  284  LOSS:  tensor(0.2718, grad_fn=<AddBackward0>)
EPOCH:  285  LOSS:  tensor(0.2702, grad_fn=<AddBackward0>)
EPOCH:  286  LOSS:  tensor(0.2685, grad_fn=<AddBackward0>)
EPOCH:  287  LOSS:  tensor(0.2668, grad_fn=<AddBackward0

EPOCH:  410  LOSS:  tensor(0.1629, grad_fn=<AddBackward0>)
EPOCH:  411  LOSS:  tensor(0.1625, grad_fn=<AddBackward0>)
EPOCH:  412  LOSS:  tensor(0.1621, grad_fn=<AddBackward0>)
EPOCH:  413  LOSS:  tensor(0.1617, grad_fn=<AddBackward0>)
EPOCH:  414  LOSS:  tensor(0.1613, grad_fn=<AddBackward0>)
EPOCH:  415  LOSS:  tensor(0.1610, grad_fn=<AddBackward0>)
EPOCH:  416  LOSS:  tensor(0.1606, grad_fn=<AddBackward0>)
EPOCH:  417  LOSS:  tensor(0.1602, grad_fn=<AddBackward0>)
EPOCH:  418  LOSS:  tensor(0.1599, grad_fn=<AddBackward0>)
EPOCH:  419  LOSS:  tensor(0.1595, grad_fn=<AddBackward0>)
EPOCH:  420  LOSS:  tensor(0.1591, grad_fn=<AddBackward0>)
EPOCH:  421  LOSS:  tensor(0.1587, grad_fn=<AddBackward0>)
EPOCH:  422  LOSS:  tensor(0.1584, grad_fn=<AddBackward0>)
EPOCH:  423  LOSS:  tensor(0.1581, grad_fn=<AddBackward0>)
EPOCH:  424  LOSS:  tensor(0.1577, grad_fn=<AddBackward0>)
EPOCH:  425  LOSS:  tensor(0.1573, grad_fn=<AddBackward0>)
EPOCH:  426  LOSS:  tensor(0.1571, grad_fn=<AddBackward0

EPOCH:  549  LOSS:  tensor(0.1284, grad_fn=<AddBackward0>)
EPOCH:  550  LOSS:  tensor(0.1282, grad_fn=<AddBackward0>)
EPOCH:  551  LOSS:  tensor(0.1280, grad_fn=<AddBackward0>)
EPOCH:  552  LOSS:  tensor(0.1279, grad_fn=<AddBackward0>)
EPOCH:  553  LOSS:  tensor(0.1277, grad_fn=<AddBackward0>)
EPOCH:  554  LOSS:  tensor(0.1276, grad_fn=<AddBackward0>)
EPOCH:  555  LOSS:  tensor(0.1274, grad_fn=<AddBackward0>)
EPOCH:  556  LOSS:  tensor(0.1273, grad_fn=<AddBackward0>)
EPOCH:  557  LOSS:  tensor(0.1271, grad_fn=<AddBackward0>)
EPOCH:  558  LOSS:  tensor(0.1270, grad_fn=<AddBackward0>)
EPOCH:  559  LOSS:  tensor(0.1268, grad_fn=<AddBackward0>)
EPOCH:  560  LOSS:  tensor(0.1267, grad_fn=<AddBackward0>)
EPOCH:  561  LOSS:  tensor(0.1265, grad_fn=<AddBackward0>)
EPOCH:  562  LOSS:  tensor(0.1264, grad_fn=<AddBackward0>)
EPOCH:  563  LOSS:  tensor(0.1262, grad_fn=<AddBackward0>)
EPOCH:  564  LOSS:  tensor(0.1260, grad_fn=<AddBackward0>)
EPOCH:  565  LOSS:  tensor(0.1259, grad_fn=<AddBackward0

...

EPOCH:  427  LOSS:  tensor(0.1594, grad_fn=<AddBackward0>)
EPOCH:  428  LOSS:  tensor(0.1590, grad_fn=<AddBackward0>)
EPOCH:  429  LOSS:  tensor(0.1587, grad_fn=<AddBackward0>)
EPOCH:  430  LOSS:  tensor(0.1585, grad_fn=<AddBackward0>)
EPOCH:  431  LOSS:  tensor(0.1582, grad_fn=<AddBackward0>)
EPOCH:  432  LOSS:  tensor(0.1579, grad_fn=<AddBackward0>)
EPOCH:  433  LOSS:  tensor(0.1577, grad_fn=<AddBackward0>)
EPOCH:  434  LOSS:  tensor(0.1574, grad_fn=<AddBackward0>)
EPOCH:  435  LOSS:  tensor(0.1572, grad_fn=<AddBackward0>)
EPOCH:  436  LOSS:  tensor(0.1569, grad_fn=<AddBackward0>)
EPOCH:  437  LOSS:  tensor(0.1567, grad_fn=<AddBackward0>)
EPOCH:  438  LOSS:  tensor(0.1564, grad_fn=<AddBackward0>)
EPOCH:  439  LOSS:  tensor(0.1562, grad_fn=<AddBackward0>)
EPOCH:  440  LOSS:  tensor(0.1560, grad_fn=<AddBackward0>)
EPOCH:  441  LOSS:  tensor(0.1557, grad_fn=<AddBackward0>)
EPOCH:  442  LOSS:  tensor(0.1554, grad_fn=<AddBackward0>)
EPOCH:  443  LOSS:  tensor(0.1552, grad_fn=<AddBackward0

EPOCH:  566  LOSS:  tensor(0.1272, grad_fn=<AddBackward0>)
EPOCH:  567  LOSS:  tensor(0.1269, grad_fn=<AddBackward0>)
EPOCH:  568  LOSS:  tensor(0.1265, grad_fn=<AddBackward0>)
EPOCH:  569  LOSS:  tensor(0.1262, grad_fn=<AddBackward0>)
EPOCH:  570  LOSS:  tensor(0.1259, grad_fn=<AddBackward0>)
EPOCH:  571  LOSS:  tensor(0.1257, grad_fn=<AddBackward0>)
EPOCH:  572  LOSS:  tensor(0.1255, grad_fn=<AddBackward0>)
EPOCH:  573  LOSS:  tensor(0.1253, grad_fn=<AddBackward0>)
EPOCH:  574  LOSS:  tensor(0.1251, grad_fn=<AddBackward0>)
EPOCH:  575  LOSS:  tensor(0.1249, grad_fn=<AddBackward0>)
EPOCH:  576  LOSS:  tensor(0.1248, grad_fn=<AddBackward0>)
EPOCH:  577  LOSS:  tensor(0.1246, grad_fn=<AddBackward0>)
EPOCH:  578  LOSS:  tensor(0.1244, grad_fn=<AddBackward0>)
EPOCH:  579  LOSS:  tensor(0.1242, grad_fn=<AddBackward0>)
EPOCH:  580  LOSS:  tensor(0.1240, grad_fn=<AddBackward0>)
EPOCH:  581  LOSS:  tensor(0.1238, grad_fn=<AddBackward0>)
EPOCH:  582  LOSS:  tensor(0.1237, grad_fn=<AddBackward0

EPOCH:  705  LOSS:  tensor(0.1035, grad_fn=<AddBackward0>)
EPOCH:  706  LOSS:  tensor(0.1033, grad_fn=<AddBackward0>)
EPOCH:  707  LOSS:  tensor(0.1032, grad_fn=<AddBackward0>)
EPOCH:  708  LOSS:  tensor(0.1030, grad_fn=<AddBackward0>)
EPOCH:  709  LOSS:  tensor(0.1029, grad_fn=<AddBackward0>)
EPOCH:  710  LOSS:  tensor(0.1027, grad_fn=<AddBackward0>)
EPOCH:  711  LOSS:  tensor(0.1026, grad_fn=<AddBackward0>)
EPOCH:  712  LOSS:  tensor(0.1024, grad_fn=<AddBackward0>)
EPOCH:  713  LOSS:  tensor(0.1023, grad_fn=<AddBackward0>)
EPOCH:  714  LOSS:  tensor(0.1022, grad_fn=<AddBackward0>)
EPOCH:  715  LOSS:  tensor(0.1020, grad_fn=<AddBackward0>)
EPOCH:  716  LOSS:  tensor(0.1019, grad_fn=<AddBackward0>)
EPOCH:  717  LOSS:  tensor(0.1018, grad_fn=<AddBackward0>)
EPOCH:  718  LOSS:  tensor(0.1016, grad_fn=<AddBackward0>)
EPOCH:  719  LOSS:  tensor(0.1015, grad_fn=<AddBackward0>)
EPOCH:  720  LOSS:  tensor(0.1014, grad_fn=<AddBackward0>)
EPOCH:  721  LOSS:  tensor(0.1013, grad_fn=<AddBackward0

EPOCH:  844  LOSS:  tensor(0.0784, grad_fn=<AddBackward0>)
EPOCH:  845  LOSS:  tensor(0.0783, grad_fn=<AddBackward0>)
EPOCH:  846  LOSS:  tensor(0.0781, grad_fn=<AddBackward0>)
EPOCH:  847  LOSS:  tensor(0.0780, grad_fn=<AddBackward0>)
EPOCH:  848  LOSS:  tensor(0.0779, grad_fn=<AddBackward0>)
EPOCH:  849  LOSS:  tensor(0.0777, grad_fn=<AddBackward0>)
EPOCH:  850  LOSS:  tensor(0.0776, grad_fn=<AddBackward0>)
EPOCH:  851  LOSS:  tensor(0.0774, grad_fn=<AddBackward0>)
EPOCH:  852  LOSS:  tensor(0.0773, grad_fn=<AddBackward0>)
EPOCH:  853  LOSS:  tensor(0.0771, grad_fn=<AddBackward0>)
EPOCH:  854  LOSS:  tensor(0.0770, grad_fn=<AddBackward0>)
EPOCH:  855  LOSS:  tensor(0.0769, grad_fn=<AddBackward0>)
EPOCH:  856  LOSS:  tensor(0.0768, grad_fn=<AddBackward0>)
EPOCH:  857  LOSS:  tensor(0.0766, grad_fn=<AddBackward0>)
EPOCH:  858  LOSS:  tensor(0.0765, grad_fn=<AddBackward0>)
EPOCH:  859  LOSS:  tensor(0.0764, grad_fn=<AddBackward0>)
EPOCH:  860  LOSS:  tensor(0.0763, grad_fn=<AddBackward0

KeyboardInterrupt: 